Code that finds a stable solution (if one exists) for arbitrary conditions (number of players vs. fronts, number of types). This code reproduces results for Section 5 (more fronts than agents), but also also reproduce results for Section 4 (more agents than fronts). This version of the code is significantly slower than Sec_4_more_players_than_fronts.ipynb. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from more_itertools import set_partitions
import pickle
import cProfile
import pstats
import time
from math import ceil, floor
import json

In [2]:
def arrangement_cost(player_biases = [0, 0], weights = [0.5, 0.5],  
                     item_label_frame = pd.DataFrame(1, index = range(2), columns = range(2)), 
                     unlabeled_cost = 1000, outcome_rule = 'mean'):
    '''
    Calculate cost of players in an arrangement. 
    
    Args:
        player_biases: dataframe of biases of each player type
        weights: dataframe of weights all players put on each front.  
        item_label_frame: dataframe of where player is competing on each front
        unlabeled_cost: cost of leaving an instance unlabeled.  
        outcome_rule: rule for outcome of the front (median vs. mean)
        
    Returns:
        dataframe of costs for all player types
    
    '''
    
    n_type = len(player_biases)
    cost_store = [0] * n_type
    M = len(weights) # number of fronts
    outcome_store = [0] * M
    
    for front in range(M):
        if item_label_frame.loc[front].sum()>0: # if there is at least one agent on that front
            total_biases = np.repeat(player_biases, item_label_frame.loc[front].tolist())
            if outcome_rule == 'mean':
                outcome_store[front] = np.mean(total_biases)
            elif outcome_rule == 'median':
                outcome_store[front] = np.median(total_biases)
        
    for player_type in range(n_type):
        cost_store[player_type] = np.sum([weights[front] * np.abs(outcome - player_biases[player_type]) 
                                          if item_label_frame.loc[front].sum()>0 
                                          else weights[front]*unlabeled_cost for 
                                          front, outcome in enumerate(outcome_store) ])

    return cost_store

def calculate_Nash(player_biases = [0, 0], n_players = [2, 2], weights = [0.5, 0.5], none_empty= False, 
                   unlabeled_cost = 1000, outcome_rule = 'mean'):
    '''
    Find all NE (if any exist). Every agent is required to compete in a front (no option to abstain). 
    
    Args:
        player_biases: list of biases of each labeler.
        n_players: number of players of each type
        weights: list of weights all players put on each front. 
        none_empty: boolean. If true, only consider arrangements where all fronts have at least one agent. 
        unlabeled_cost: cost of leaving an instance unlabeled.  
        outcome_rule: rule for outcome of the front (median vs. mean)
        
    Returns:
        dataframe of all arrangements, cost for all involved and whether it is stable
    
    '''
    
    n_type = len(player_biases)
    M = len(weights) # number of fronts
        
    # list of length n_type, giving all possible arrangements of each type
    arrange_by_type = [[part for part in itertools.combinations_with_replacement(range(M), n_players[i])]
                       for i in range(n_type)]
    # all possible arrangements of all types
    all_combinations = list(itertools.product(*arrange_by_type))
    if none_empty:
        # only consider arrangements where all fronts have at least one agent
        all_combinations = [comb_val for comb_val in all_combinations
                       if np.all([str(mi) in str(comb_val) for mi in range(M)])]
        if len(all_combinations) ==0:
            raise Exception('Not enough agents to cover all fronts')
    # store costs for each player type
    store_costs = pd.DataFrame(index = range(len(list(all_combinations))), 
                               columns = range(n_type))
    # reverse dictionary for arrangements
    rev_dict = dict(zip(all_combinations, range(len(all_combinations))))
    
    # calculate error rate experienced by different players    
    for arrange_i, arrangement in enumerate(all_combinations):
        # make dataframe of player locations
        item_label_frame = pd.DataFrame(0, index = range(M), columns = range(n_type))
        for players_type, locs in enumerate(arrangement):
            for loc in locs: # go through locations of all players of this type
                item_label_frame.iloc[loc, players_type] += 1
        store_costs.iloc[arrange_i, :] = arrangement_cost(player_biases = player_biases, weights = weights, 
                                                          item_label_frame = item_label_frame, 
                                                          outcome_rule = outcome_rule, 
                                                          unlabeled_cost = unlabeled_cost)
        
    store_costs['arrange'] = [str(comb) for comb in all_combinations]
    # calculate which arrangements are stable
    store_costs['stable'] = True #default to all arrangements being stable
    for arrange_loc, arrangement in enumerate(all_combinations): # all combinations
        for players_type, locs in enumerate(arrangement): # all groups of player types
            for player_loc in list(np.unique(locs)): # consider unique players where the player type is
                for item_val in range(M): # all alternative locations
                    if item_val != player_loc:
                        new_list =  list(locs)
                        new_list.remove(player_loc)
                        arrangement_copy = list(arrangement)
                        arrangement_copy[players_type] = tuple(sorted([item_val] + new_list))
                        # if none_empty = True, then can't move to arrangements that would leave fronts empty
                        if tuple(arrangement_copy) in all_combinations: 
                            new_arrange_loc = rev_dict[tuple(arrangement_copy)]
                            if (store_costs.loc[new_arrange_loc, players_type]< 
                                store_costs.loc[arrange_loc, players_type]):
                                # if lower cost, this proves arrangement isn't stable. 
                                store_costs.loc[arrange_loc, 'stable'] = str(tuple(arrangement_copy)) 
                                break 

    return store_costs

Illustrating results in Section 5 (cases where no stable arrangement exists). 

In [3]:
# Illustrating Lemma 12 (parameters where no stable arrangement exists for median outcome)
n_front = 4
n_players = 3 # need n_players < n_front
store_costs = calculate_Nash(player_biases = [1, -0.5], n_players = [1, n_players-1], 
                             weights = [1/n_front] * n_front, unlabeled_cost = 0.3, outcome_rule = 'median')
np.any(store_costs['stable'] == True)

False

In [4]:
# Illustrating Lemma 13 (parameters where no stable arrangement exists for mean outcome)
n_front = 5
n_players = 4 # need n_players < n_front
store_costs = calculate_Nash(player_biases = [1, -0.5], n_players = [1, n_players-1], 
                             weights = [1/n_front] * n_front, unlabeled_cost = 0.2, outcome_rule = 'mean')
np.any(store_costs['stable'] == True)

False

In [12]:
# Illustrating Lemma 14 (for 3 players, at least 4 fronts, arbitrary player biases, mean outcome, there is always
# a stable arrangement)
nsim = 10
n_front = 4
for _ in range(nsim):
    store_costs = calculate_Nash(player_biases = np.random.rand(3).tolist(), n_players = [1] * 3, 
                                 weights = [1] * 3, unlabeled_cost = np.abs(np.random.rand(1)[0]), 
                                 outcome_rule = 'mean')
    if np.any(store_costs['stable'] == True) == False: 
        print('no stable arrangement exists')
    else:
        # print which arrangement(s) are stable
        # notation: (0,), (1,), (1,) means that player 0 is on front 0, and players 1 and 2 are on front 1
        print([eval(arrange) for arrange in store_costs['arrange'][store_costs['stable'] == True].tolist()])

[((0,), (1,), (2,)), ((0,), (2,), (1,)), ((1,), (0,), (2,)), ((1,), (2,), (0,)), ((2,), (0,), (1,)), ((2,), (1,), (0,))]
[((0,), (1,), (2,)), ((0,), (2,), (1,)), ((1,), (0,), (2,)), ((1,), (2,), (0,)), ((2,), (0,), (1,)), ((2,), (1,), (0,))]
[((0,), (1,), (2,)), ((0,), (2,), (1,)), ((1,), (0,), (2,)), ((1,), (2,), (0,)), ((2,), (0,), (1,)), ((2,), (1,), (0,))]
[((0,), (1,), (2,)), ((0,), (2,), (1,)), ((1,), (0,), (2,)), ((1,), (2,), (0,)), ((2,), (0,), (1,)), ((2,), (1,), (0,))]
[((0,), (1,), (0,)), ((0,), (2,), (0,)), ((1,), (0,), (1,)), ((1,), (2,), (1,)), ((2,), (0,), (2,)), ((2,), (1,), (2,))]
[((0,), (1,), (0,)), ((0,), (2,), (0,)), ((1,), (0,), (1,)), ((1,), (2,), (1,)), ((2,), (0,), (2,)), ((2,), (1,), (2,))]
[((0,), (1,), (2,)), ((0,), (2,), (1,)), ((1,), (0,), (2,)), ((1,), (2,), (0,)), ((2,), (0,), (1,)), ((2,), (1,), (0,))]
[((0,), (1,), (2,)), ((0,), (2,), (1,)), ((1,), (0,), (2,)), ((1,), (2,), (0,)), ((2,), (0,), (1,)), ((2,), (1,), (0,))]
[((0,), (1,), (2,)), ((0,), (2,)